# Визначте DataFrame з тривимірними векторами слів

In [88]:
import pickle
from pathlib import Path

import pandas as pd

word_embeddings = Path.cwd() / "data" / "word_embeddings_subset.p"

with word_embeddings.open("rb") as file:
    word_embeddings = pickle.load(file)

In [89]:
word_embeddings.keys()

dict_keys(['country', 'city', 'China', 'Iraq', 'oil', 'town', 'Canada', 'London', 'England', 'Australia', 'Japan', 'Pakistan', 'Iran', 'gas', 'happy', 'Russia', 'Afghanistan', 'France', 'Germany', 'Georgia', 'Baghdad', 'village', 'Spain', 'Italy', 'Beijing', 'Jordan', 'Paris', 'Ireland', 'Turkey', 'Egypt', 'Lebanon', 'Taiwan', 'Tokyo', 'Nigeria', 'Vietnam', 'Moscow', 'Greece', 'Indonesia', 'sad', 'Syria', 'Thailand', 'Libya', 'Zimbabwe', 'Cuba', 'Ottawa', 'Tehran', 'Sudan', 'Kenya', 'Philippines', 'Sweden', 'Poland', 'Ukraine', 'Rome', 'Venezuela', 'Switzerland', 'Berlin', 'Bangladesh', 'Portugal', 'Ghana', 'Athens', 'king', 'Madrid', 'Somalia', 'Dublin', 'Qatar', 'Chile', 'Islamabad', 'Bahrain', 'Nepal', 'Norway', 'Serbia', 'Kabul', 'continent', 'Brussels', 'Belgium', 'Uganda', 'petroleum', 'Cairo', 'Denmark', 'Austria', 'Jamaica', 'Georgetown', 'Bangkok', 'Finland', 'Peru', 'Romania', 'Bulgaria', 'Hungary', 'Vienna', 'Kingston', 'Manila', 'Cyprus', 'Azerbaijan', 'Copenhagen', 'Fiji',

# Визначте функції для пошуку найближчого слова

In [90]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

def return_3d(arr: np.ndarray) -> np.ndarray:
    # Гарантуємо 2D float масив
    A = np.asarray(arr, dtype=np.float64)
    if A.ndim != 2:
        raise ValueError(f"PCA очікує 2D масив (n_samples, n_features), отримано shape={A.shape}")
    n_samples, n_features = A.shape
    if n_samples < 1 or n_features < 1:
        raise ValueError("Порожній масив для PCA.")
    n_components = min(3, n_samples, n_features)
    pca = PCA(n_components=n_components)
    return pca.fit_transform(A)

def dict_to_db(dct: dict) -> pd.DataFrame:
    # Перетворюємо словник {слово: вектор} у 2D числовий масив
    if not isinstance(dct, dict) or len(dct) == 0:
        raise ValueError("Очікується непорожній словник ембеддінгів.")
    keys = list(dct.keys())
    vecs = [np.asarray(v, dtype=np.float64).ravel() for v in dct.values()]
    lengths = {v.size for v in vecs}
    if len(lengths) != 1:
        raise ValueError(f"Всі вектори мають бути однакової довжини, отримано довжини: {sorted(lengths)}")
    X = np.stack(vecs, axis=0)  # (n_samples, n_features)
    Xp = return_3d(X)
    cols = [f"pc{i+1}" for i in range(Xp.shape[1])]
    return pd.DataFrame(data=Xp, index=keys, columns=cols)

def vec(word: str, embeddings: pd.DataFrame) -> np.ndarray:
    try:
        return embeddings.loc[word].to_numpy(dtype=np.float64)
    except KeyError as e:
        raise KeyError(f"Слова '{word}' немає в ембеддінгах (індексі DataFrame).") from e

def _row_normalize(M: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    # L2-нормалізація рядків
    norms = np.linalg.norm(M, axis=1, keepdims=True)
    norms = np.maximum(norms, eps)
    return M / norms

def find_closest_word(df: pd.DataFrame, word_vec: np.ndarray, exclude: set[str] | None = None):
    """
    Шукає найближче слово за косинусною подібністю у 3D-просторі df.
    exclude – множина слів, які не можна повертати (напр., початкові).
    """
    if exclude is None:
        exclude = set()
    # Матриця усіх векторів (n_words, n_dims)
    M = df.to_numpy(dtype=np.float64)
    # Нормалізуємо матрицю та запит
    U = _row_normalize(M)
    q = np.asarray(word_vec, dtype=np.float64).ravel()
    q_norm = q / max(np.linalg.norm(q), 1e-12)
    # Косинусні подібності
    sims = U @ q_norm  # (n_words,)
    # Відкинути заборонені індекси
    mask = np.ones(len(df.index), dtype=bool)
    if exclude:
        exclude_idx = [i for i, w in enumerate(df.index) if w in exclude]
        mask[np.array(exclude_idx, dtype=int)] = False
    # argmax по дозволених
    allowed_sims = np.where(mask, sims, -np.inf)
    i = int(np.argmax(allowed_sims))
    return df.index[i]

def find_topk_words(df: pd.DataFrame, word_vec: np.ndarray, k: int = 5, exclude: set[str] | None = None):
    """
    Повертає top-k найближчих слів за косинусом.
    """
    if exclude is None:
        exclude = set()
    M = df.to_numpy(dtype=np.float64)
    U = _row_normalize(M)
    q = np.asarray(word_vec, dtype=np.float64).ravel()
    q_norm = q / max(np.linalg.norm(q), 1e-12)
    sims = U @ q_norm
    # Відкидаємо exclude
    pairs = [(w, s) for w, s in zip(df.index.tolist(), sims.tolist()) if w not in exclude]
    pairs.sort(key=lambda x: x[1], reverse=True)
    return pairs[:k]

## Деякі приклади використання функції пошуку

In [93]:
df = dict_to_db(word_embeddings)
# Класична аналогія: capital_offset = Paris - France
capital = vec('Paris', df) - vec('France', df)
# Приклад: China + (Paris - France) ≈ Beijing
target = vec('China', df) + capital
print(find_closest_word(df=df, word_vec=target, exclude={'China', 'Paris', 'France'}))
# Також подивимось top-5
print(find_topk_words(df=df, word_vec=target, k=5, exclude={'China', 'Paris', 'France'}))

London
[('London', 0.9929758981539933), ('Tokyo', 0.9780659847333701), ('Montevideo', 0.9683353580080912), ('city', 0.9645896406653922), ('Dublin', 0.96246823502562)]


In [95]:
# Приклад: England + (Paris - France) ≈ London
target = vec('England', df) + capital
print(find_closest_word(df=df, word_vec=target, exclude={'China', 'Paris', 'France'}))
# Також подивимось top-5
print(find_topk_words(df=df, word_vec=target, k=5, exclude={'China', 'Paris', 'France'}))

London
[('London', 0.9929758981539933), ('Tokyo', 0.9780659847333701), ('Montevideo', 0.9683353580080912), ('city', 0.9645896406653922), ('Dublin', 0.96246823502562)]


In [96]:
target = vec("oil", df) - vec("king", df)

print(find_closest_word(df=df, word_vec=target, exclude={'China', 'Paris', 'France'}))
# Також подивимось top-5
print(find_topk_words(df=df, word_vec=target, k=5, exclude={'China', 'Paris', 'France'}))

Kyrgyzstan
[('Kyrgyzstan', 0.9014647405248613), ('Tajikistan', 0.8970380281526311), ('Eritrea', 0.8606097712180828), ('Turkmenistan', 0.8231562406378717), ('Burundi', 0.7646099109258514)]


# Обчисліть векторний добуток для знаходження ортогонального слова

In [98]:
ort1 = np.cross(vec("oil", df), vec("king", df))

print(find_closest_word(df=df, word_vec=ort1, exclude={'oil', 'king'}))
# Також подивимось top-5
print(find_topk_words(df=df, word_vec=ort1, k=5, exclude={'oil', 'king'}))

Georgia
[('Georgia', 0.9443204394917163), ('Estonia', 0.9423309900125145), ('Lithuania', 0.9401424418884858), ('Poland', 0.9377380907370603), ('Hungary', 0.9258560318606222)]


In [100]:
ort2 = np.cross(vec("France", df), vec("Paris", df))

print(find_closest_word(df=df, word_vec=ort2, exclude={'France', 'Paris'}))
# Також подивимось top-5
print(find_topk_words(df=df, word_vec=ort2, k=5, exclude={'France', 'Paris'}))

Zagreb
[('Zagreb', 0.9210633044760501), ('Skopje', 0.9074607442877155), ('Moscow', 0.8962049551696625), ('Vaduz', 0.8927146550044899), ('Riga', 0.8890936701109639)]


# Напишіть функції визначення кута між словами

In [104]:
import numpy as np

def vec_from_dict(word: str, embeddings: dict[str, np.ndarray]) -> np.ndarray:
    return embeddings[word]

def angle_between(u: np.ndarray, v: np.ndarray) -> float:
    u = np.asarray(u, dtype=np.float64).ravel()
    v = np.asarray(v, dtype=np.float64).ravel()
    nu = np.linalg.norm(u)
    nv = np.linalg.norm(v)
    if nu == 0 or nv == 0:
        raise ValueError("Неможливо обчислити кут для нульового вектора.")
    cos_theta = float(np.dot(u, v) / (nu * nv))
    # Числова стабільність
    cos_theta = max(-1.0, min(1.0, cos_theta))
    theta = np.arccos(cos_theta)
    return float(np.degrees(theta))

## Використаємо функцію

In [105]:
country = vec_from_dict("France", word_embeddings)
capital = vec_from_dict("Paris", word_embeddings)

print(angle_between(country, capital))

50.69184994088394


In [106]:
oil = vec_from_dict("oil", word_embeddings)
king = vec_from_dict("king", word_embeddings)
print(angle_between(oil, king))

86.48644951540895


In [108]:
france = vec_from_dict("France", word_embeddings)
england = vec_from_dict('England', word_embeddings)

print(angle_between(france, england))

66.54370256990977


## Висновки щодо кутів між словами

З практичних прикладів випливає, що чим ближчі слова одне до одного, тим менший кут між ними. Із загального: стиснення знижує точність знаходження як найближчого слова, так і кутів між словами.